<a href="https://colab.research.google.com/github/IraitzM/dlt_homeworks/blob/main/Workshop_Part1_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Homework

Write a python script that will load data from the SpaceX API into DuckDB using dlt.

Use:
- @dlt.source
- @dlt.resource
- @dlt.transformer

SpaceX API URL: https://api.spacexdata.com

Docs: https://github.com/r-spacex/SpaceX-API/blob/master/docs/README.md

Endpoints for loading:
- launches
- rockets
- crew

# Install dlt with duckdb extention

In [1]:
%%capture
!pip install dlt[duckdb]

# Play with SpaceX API

In [2]:
import requests
response = requests.get("https://api.spacexdata.com/v4/launches")
response.json()[0]

{'fairings': {'reused': False,
  'recovery_attempt': False,
  'recovered': False,
  'ships': []},
 'links': {'patch': {'small': 'https://images2.imgbox.com/94/f2/NN6Ph45r_o.png',
   'large': 'https://images2.imgbox.com/5b/02/QcxHUb5V_o.png'},
  'reddit': {'campaign': None,
   'launch': None,
   'media': None,
   'recovery': None},
  'flickr': {'small': [], 'original': []},
  'presskit': None,
  'webcast': 'https://www.youtube.com/watch?v=0a_00nJ_Y88',
  'youtube_id': '0a_00nJ_Y88',
  'article': 'https://www.space.com/2196-spacex-inaugural-falcon-1-rocket-lost-launch.html',
  'wikipedia': 'https://en.wikipedia.org/wiki/DemoSat'},
 'static_fire_date_utc': '2006-03-17T00:00:00.000Z',
 'static_fire_date_unix': 1142553600,
 'net': False,
 'window': 0,
 'rocket': '5e9d0d95eda69955f709d1eb',
 'success': False,
 'failures': [{'time': 33,
   'altitude': None,
   'reason': 'merlin engine failure'}],
 'details': 'Engine failure at 33 seconds and loss of vehicle',
 'crew': [],
 'ships': [],
 'caps

# Helper
Run the cell and ignore it.

In [4]:
from dlt.common.pipeline import LoadInfo

def assert_load_info(info: LoadInfo, expected_load_packages: int = 1) -> None:
    """Asserts that expected number of packages was loaded and there are no failed jobs"""
    assert len(info.loads_ids) == expected_load_packages
    # all packages loaded
    assert all(package.state == "loaded" for package in info.load_packages) is True
    # no failed jobs in any of the packages
    info.raise_on_failed_jobs()

# Task 1


Create a pipeline for SpaceX API, for the next endpoints: launches, rockets, crew.

- Fill the empty lines in the functions below.
- `get_rockets` resource should have `table_name=rockets`.
- Create a [resource](https://dlthub.com/docs/general-usage/resource#declare-a-resource) for the `crew` endpoint from scratch.
- [Run the pipeline](https://dlthub.com/docs/walkthroughs/run-a-pipeline) without errors.

In [7]:
import time

import dlt
import requests


@dlt.resource(table_name="launches")
def get_launches():
    # put your code here
    response = requests.get("https://api.spacexdata.com/v4/launches")
    resp_json = response.json()
    for rjson in resp_json:
      yield rjson

@dlt.resource(table_name="rockets")
def get_rockets():
    # put your code here
    response = requests.get("https://api.spacexdata.com/v4/rockets")
    resp_json = response.json()
    for rjson in resp_json:
      yield rjson


@dlt.resource(table_name="crew")
def get_crew():
  # put your code here
  response = requests.get("https://api.spacexdata.com/v4/crew")
  resp_json = response.json()
  for rjson in resp_json:
    yield rjson


pipeline = dlt.pipeline(
    pipeline_name='spacex_with_source',
    destination='duckdb',
    dataset_name='spacex_data',
    dev_mode=True,
)

load_info = pipeline.run([get_launches(), get_rockets(), get_crew()])
print(load_info)
assert_load_info(load_info)


Pipeline spacex_with_source load step completed in 1.28 seconds
1 load package(s) were loaded to destination duckdb and into dataset spacex_data_20240822040555
The duckdb destination used duckdb:////content/spacex_with_source.duckdb location to store data
Load package 1724342755.9962318 is LOADED and contains no failed jobs


Run the code below and
## Answer the Question:
- What weight (kg) has the heighest (meters) rocket?

In [8]:
import duckdb
from google.colab import data_table
data_table.enable_dataframe_formatter()

# a database '<pipeline_name>.duckdb' was created in working directory so just connect to it
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
stats_table = conn.sql("SELECT * FROM rockets").df()
display(stats_table)

,height__meters,height__feet,diameter__meters,diameter__feet,mass__kg,mass__lb,first_stage__thrust_sea_level__k_n,first_stage__thrust_sea_level__lbf,first_stage__thrust_vacuum__k_n,first_stage__thrust_vacuum__lbf,...,country,company,wikipedia,description,id,_dlt_load_id,_dlt_id,height__feet__v_double,engines__thrust_to_weight__v_double,landing_legs__material
0,22.25,73.0,1.68,5.5,30146,66460,420,94000,480,110000,...,Republic of the Marshall Islands,SpaceX,https://en.wikipedia.org/wiki/Falcon_1,The Falcon 1 was an expendable launch system p...,5e9d0d95eda69955f709d1eb,1724342755.9962318,quiB1brcmamaJA,NaN,NaN,None
1,70.00,NaN,3.70,12.0,549054,1207920,7607,1710000,8227,1849500,...,United States,SpaceX,https://en.wikipedia.org/wiki/Falcon_9,Falcon 9 is a two-stage rocket designed and ma...,5e9d0d95eda69973a809d1ec,1724342755.9962318,oQnyqS/w2Fpx6g,229.6,180.1,carbon fiber
2,70.00,NaN,12.20,39.9,1420788,3125735,22819,5130000,24681,5548500,...,United States,SpaceX,https://en.wikipedia.org/wiki/Falcon_Heavy,With the ability to lift into orbit over 54 me...,5e9d0d95eda69974db09d1ed,1724342755.9962318,KbXiQJM9pDpuQQ,229.6,180.1,carbon fiber
3,118.00,387.0,9.00,30.0,1335000,2943000,128000,28775544,138000,31023634,...,United States,SpaceX,https://en.wikipedia.org/wiki/SpaceX_Starship,Starship and Super Heavy Rocket represent a fu...,5e9d0d96eda699382d09d1ee,1724342755.9962318,uq518VsZX9N13A,NaN,NaN,stainless steel


In [9]:
stats_table.columns

Index(['height__meters', 'height__feet', 'diameter__meters', 'diameter__feet',
       'mass__kg', 'mass__lb', 'first_stage__thrust_sea_level__k_n',
       'first_stage__thrust_sea_level__lbf', 'first_stage__thrust_vacuum__k_n',
       'first_stage__thrust_vacuum__lbf', 'first_stage__reusable',
       'first_stage__engines', 'first_stage__fuel_amount_tons',
       'first_stage__burn_time_sec', 'second_stage__thrust__k_n',
       'second_stage__thrust__lbf',
       'second_stage__payloads__composite_fairing__height__meters',
       'second_stage__payloads__composite_fairing__height__feet',
       'second_stage__payloads__composite_fairing__diameter__meters',
       'second_stage__payloads__composite_fairing__diameter__feet',
       'second_stage__payloads__option_1', 'second_stage__reusable',
       'second_stage__engines', 'second_stage__fuel_amount_tons',
       'second_stage__burn_time_sec', 'engines__isp__sea_level',
       'engines__isp__vacuum', 'engines__thrust_sea_level__k_n',
  

# Task 2
- Add pagination, read [SpaceX API doc](https://github.com/r-spacex/SpaceX-API/blob/master/docs/queries.md).
- Combine all resources in one [source](https://dlthub.com/docs/general-usage/source) and the pipeline with `@dlt.source`.
- Add incremental loading for resource `get_launches` using `merge` write disposition, `id` as a  primary key and `dlt.sources.incremental`.
- Run the pipeline [only with](https://dlthub.com/docs/general-usage/source#access-and-select-resources-to-load) `get_launches` resource.

Read more about [incremental loading](https://dlthub.com/docs/general-usage/incremental-loading).

## Try post method to query SpaceX API


In [28]:
import requests

response = requests.post(
    "https://api.spacexdata.com/v4/launches/query",
    json={
        "query": {
            "date_utc": {
                "$gt": "2022-11-01 00:00:00+00:00",
                "$lt": "2022-11-02 00:00:00+00:00",
            },
        },
        "options": {
            "page": 1
        }
    }
)
response.json().keys()

dict_keys(['docs', 'totalDocs', 'limit', 'totalPages', 'page', 'pagingCounter', 'hasPrevPage', 'hasNextPage', 'prevPage', 'nextPage'])

In [29]:
response.json()

{'docs': [{'fairings': {'reused': None,
    'recovery_attempt': None,
    'recovered': None,
    'ships': []},
   'links': {'patch': {'small': None, 'large': None},
    'reddit': {'campaign': None,
     'launch': None,
     'media': None,
     'recovery': None},
    'flickr': {'small': [], 'original': []},
    'presskit': None,
    'webcast': 'https://youtu.be/pY628jRd6gM',
    'youtube_id': 'pY628jRd6gM',
    'article': None,
    'wikipedia': None},
   'static_fire_date_utc': None,
   'static_fire_date_unix': None,
   'net': False,
   'window': None,
   'rocket': '5e9d0d95eda69974db09d1ed',
   'success': None,
   'failures': [],
   'details': None,
   'crew': [],
   'ships': [],
   'capsules': [],
   'payloads': ['5fe3b86eb3467846b324217c'],
   'launchpad': '5e9e4502f509094188566f88',
   'flight_number': 188,
   'name': 'USSF-44',
   'date_utc': '2022-11-01T13:41:00.000Z',
   'date_unix': 1667310060,
   'date_local': '2022-11-01T09:41:00-04:00',
   'date_precision': 'hour',
   'upcomi

Use code above to make your launches resource incremental. Use date_unix 0 as an initial value.

In [24]:
import dlt
import requests

def pagination(date_unix, page = 0):
    while True:
        response = requests.post(
            "https://api.spacexdata.com/v4/launches/query",
            json={
                "query": {
                    "date_utc": {
                        "$gt": date_unix,
                    },
                },
                "options": {
                    "page": page
                }
            }
        )
        response.raise_for_status()
        yield response.json()         # Here we're yielding pages

        # Get next page
        if "nextPage" not in response.json():
            break
        page = response.json()["nextPage"]

@dlt.resource(
    table_name="lauches"
)
def get_launches(date_unix="2022-11-01 00:00:00+00:00"):
    # put your code here
    yield pagination(date_unix)


@dlt.resource(table_name="rockets")
def get_rockets():
    # put your code here
    response = requests.get("https://api.spacexdata.com/v4/rockets")
    resp_json = response.json()
    for rjson in resp_json:
      yield rjson


@dlt.resource(table_name="crew")
def get_crew():
  # put your code here
  response = requests.get("https://api.spacexdata.com/v4/crew")
  resp_json = response.json()
  for rjson in resp_json:
    yield rjson


@dlt.source
def spacex_source():
    return [get_launches, get_rockets, get_crew]


pipeline = dlt.pipeline(
    pipeline_name='spacex_with_source_inc',
    destination='duckdb',
    dataset_name='spacex_data_inc',
    dev_mode=True,
)

data = spacex_source()

load_info = pipeline.run(data)
print(load_info)
assert_load_info(load_info)

# Run the pipeline one more time, it should load no data
load_info = pipeline.run(data)
print(load_info)
assert_load_info(load_info, expected_load_packages=0)

KeyboardInterrupt: 

## Answer the Question:
What rocket was launched between 2022-11-01 and 2022-11-02?


In [21]:
import duckdb
from google.colab import data_table
data_table.enable_dataframe_formatter()

# a database '<pipeline_name>.duckdb' was created in working directory so just connect to it
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
stats_table = conn.sql("SELECT * FROM launches WHERE date_utc BETWEEN '2022-11-01 00:00:00+00:00' AND '2022-11-02 00:00:00+00:00'").df()
display(stats_table)

CatalogException: Catalog Error: SET search_path: No catalog + schema named "spacex_data_inc_20240822043448" found.

# Task 3: Get payloads of launches

Use `@dlt.transformer` to get additional info for your data.

Read more about dlt [transformers](https://dlthub.com/docs/general-usage/resource#process-resources-with-dlttransformer).

In [ ]:
import dlt
import requests


def get_launches():
    # put your code here



def get_payloads(items):
    # put your code here


pipeline = dlt.pipeline(
    pipeline_name='spacex_with_source_tr',
    destination='duckdb',
    dataset_name='spacex_data_tr',
    dev_mode=True,
)

data = get_launches | get_payloads

load_info = pipeline.run(data())
print(load_info)
assert_load_info(load_info)


In [ ]:
import duckdb
from google.colab import data_table
data_table.enable_dataframe_formatter()

# a database '<pipeline_name>.duckdb' was created in working directory so just connect to it
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
# list all tables
display(conn.sql("DESCRIBE"))

Run the code below and

## Answer the Question:

What regime has Satellite "FalconSAT-2" with launch id: 5eb87cd9ffd86e000604b32a?  

In [ ]:
stats_table = conn.sql("SELECT * FROM details").df()
display(stats_table)